# Kafka Demo

### Connect to Kafka Broker Server
```
ssh -o ServerAliveInterval=60 -L 9092:localhost:9092 tunnel@128.2.24.106 -NTf

For hw2,
ssh -o ServerAliveInterval=60 -L 9092:localhost:9092 tunnel@128.2.204.215 -NTf

```
pass: seaitunnel

Will get the following if I already did above

"""
Address already in use
channel_setup_fwd_listener_tcpip: cannot listen to port: 9092
Could not request local forwarding.
"""

Need to use conda for virtual env, not just normal python 
To kill connection at port:
```
lsof -ti:9092 | xargs kill -9
```

### Setup
```
python -m pip install kafka-python
```

In [4]:
from platform import python_version
print(python_version())

3.9.5


In [7]:
!pip3 list

Package      Version
------------ -------
kafka-python 2.0.2
pip          21.1.1
setuptools   56.0.0
You should consider upgrading via the '/Users/joonghochoi/Desktop/kafka/demo/venv/bin/python -m pip install --upgrade pip' command.


In [1]:
from os import path
import sys, os
from datetime import datetime
from json import dumps, loads
from time import sleep
from random import randint
import numpy as np
# ssh -o ServerAliveInterval=60 -L 9092:localhost:9092 tunnel@128.2.24.106 -NTf
from kafka import KafkaConsumer, KafkaProducer

# Update this for your demo otherwise you'll see my data :)
topic = 'movielog14'

### Producer Mode -> Writes Data to Broker

In [4]:
# Create a producer to write data to kafka
# Ref: https://kafka-python.readthedocs.io/en/master/apidoc/KafkaProducer.html
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                        value_serializer=lambda x: dumps(x).encode('utf-8'),
                        )
cities = ['Pittsburgh','New York','London','Bangalore','Shanghai','Tokyo','Munich']
# Write data via the producer
print("Writing to Kafka Broker")
for i in range(10):
    data = f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")},{cities[randint(0,len(cities)-1)]},{randint(18, 32)}ºC'
    print(f"Writing: {data}")
    producer.send(topic=topic, value=data)
    sleep(1)

Writing to Kafka Broker
Writing: 2023-02-07 16:01:58,New York,22ºC
Writing: 2023-02-07 16:01:59,London,28ºC
Writing: 2023-02-07 16:02:00,Bangalore,24ºC
Writing: 2023-02-07 16:02:01,London,27ºC
Writing: 2023-02-07 16:02:02,Munich,18ºC
Writing: 2023-02-07 16:02:03,Tokyo,31ºC
Writing: 2023-02-07 16:02:04,Bangalore,27ºC
Writing: 2023-02-07 16:02:05,Shanghai,19ºC
Writing: 2023-02-07 16:02:06,New York,31ºC
Writing: 2023-02-07 16:02:07,Munich,19ºC


### Consumer Mode -> Reads Data from Broker

In [ ]:
# Create a consumer to read data from kafka
# Ref: https://kafka-python.readthedocs.io/en/master/apidoc/KafkaConsumer.html
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=['localhost:9092'],
    # Read from the start of the topic; Default is latest
    auto_offset_reset='earliest', #get everything from the start
    # auto_offset_reset='latest',
    # group_id='team13',
    # Commit that an offset has been read
    enable_auto_commit=True,
    # How often to tell Kafka, an offset has been read
    auto_commit_interval_ms=1000
)

print('Reading Kafka Broker')
for message in consumer:

    message = message.value.decode()
    # Default message.value type is bytes!
    #print(loads(message))
    os.system(f"echo {message} >> kafka_log1.csv")

Reading Kafka Broker


# OR Use kcat!
It's a CLI (Command Line Interface). Previously known as kafkacat

Install with your package installer such as:
```
brew install kcat
apt-get install kcat
& more
```

Use: 
```
kcat -b <broker> -t <topic> -p <partition>
```

Ref: https://docs.confluent.io/platform/current/app-development/kafkacat-usage.html

In [7]:
! kcat -b localhost:9092 -t "movielog14" -p earliest

% Auto-selecting Consumer mode (use -P or -C to override)
% ERROR: Topic moviglog14 error: Broker: Unknown topic or partition


### See more options using
```
kcat
```

In [ ]:
!kcat -b localhost:9092 -t "movielog14" -o end | grep "request"